### **Rede Neural Multilayer Perceptron MLP (Classificação)**
* Carregamento dos pacotes necessários
* Carregamento dos dados
* Definição do modelo
* Treinamento de modelo
* Gráficos e Avaliação
* Exercício de Apoio

### **Pacotes**

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import sklearn.datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

### **Dados**
**Conjunto de dados Iris**

Este dataset contém 150 exemplos, sendo 50 de cada classe: Setosa, Virginica e Versicolor. Os exemplos
são caracterizados por quatro atributos: comprimento e largura das pétalas e sépalas
https://en.wikipedia.org/wiki/Iris_flower_data_set

### **Carga dos Dados**

In [3]:
dados = sklearn.datasets.load_iris()
# Imprimindo algumas informações sobre o conjunto de dados
print("Atributos:",dados['feature_names'])
print("Classes (labels):",dados['target_names'])
print("Dimensões:", dados['data'].shape)
df = pd.DataFrame(dados.data, columns = dados.feature_names)
df['label'] = [dados.target_names[i] for i in dados.target]
Atributos: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Classes (labels): ['setosa' 'versicolor' 'virginica']
Dimensões: (150, 4)
# Pair Plot
sns.pairplot(df, hue = 'label')

SyntaxError: ignored

### **Preprocessamento dos Dados**

In [ ]:
X = df.drop(['label'], axis=1)
# Normalização dos dados (Min-Max)
normalizador = MinMaxScaler()
X_norm = pd.DataFrame(normalizador.fit_transform(X), columns=X.columns)
labels = df.label.unique()
print(labels)
#Transformação dos atributos (classes) categóricos em
# numéricos (1-de-c / one-hot-encoding)
# setosa -> 0 0 1
# versicolor -> 0 1 0
# virginica -> 1 0 0
Y = pd.get_dummies(df.label)
print(Y)

### **Separação Treino/Validação/Teste**

In [ ]:
# Separação desenvolvimento (90) e teste (10)
X_dev, X_test, Y_dev, Y_test = train_test_split(X_norm, Y, test_size=0.1, random_state=1)
# Separação treino (80) e validação (20)
X_train, X_val, Y_train, Y_val = train_test_split(X_dev, Y_dev, test_size=0.2, random_state=1)
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

### **Modelo**

In [ ]:
class MLP(nn.Module):
def init (self,input_dim,output_dim):
super(MLP,self). init ()
self.rede = nn.Sequential(
nn.Linear(input_dim,10),
nn.Tanh(),
nn.Linear(10,output_dim),
nn.Sigmoid(),
)
def forward(self,x):
out = self.rede(x)
return out

### **Instanciação do Modelo**

In [ ]:
input_dim = 4 # número de atributos do Iris
output_dim = 3 # número de classes
modelo = MLP(input_dim,output_dim) # Criação do modelo (rede)
from torchsummary import summary
print(modelo)
summary(modelo, (150,4))
MLP(
(rede): Sequential(
(0) : Linear(in_features=4, out_features=10, bias=True)
(1) : Tanh()
(2) : Linear(in_features=10, out_features=3, bias=True)
(3) : Sigmoid()
)
)


### **Otimizador e Função de Custo**

In [ ]:
eta = 0.2
loss_function = nn.MSELoss()
optimizer = torch.optim.SGD(modelo.parameters(),lr=eta)

### **Treinamento**
### **Transformação dos dados em tensores Pytorch**

In [ ]:
x_train = torch.FloatTensor(X_train.values)
y_train = torch.FloatTensor(Y_train.values)
x_val = torch.FloatTensor(X_val.values)
y_val = torch.FloatTensor(Y_val.values)
x_test = torch.FloatTensor(X_test.values)
y_test = torch.FloatTensor(Y_test.values)
# verificando disponibilidade da gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

### **Laço de treinamento da rede**

In [ ]:
def train_network(model,optimizer,loss_function,x_train,y_train,x_val,y_val,num_epochs,train_losses,val_
for epoch in range(num_epochs):
# zerando os gradientes da época anterior
optimizer.zero_grad()
# fase de propagação
output_train = model(x_train)
# cálculo do erro (função de custo - loss function)
loss_train = loss_function(output_train, y_train)
# fase de retroprogação
loss_train.backward()
# atualização dos pesos da rede
optimizer.step()
# avaliando o modelo com o conjunto de validação
output_val = model(x_val)
loss_val = loss_function(output_val,y_val)
train_losses[epoch] = loss_train.item()
val_losses[epoch] = loss_val.item()
if (epoch + 1) % 100 == 0:
print(f"Epoch {epoch+1}/{num_epochs}, Erro Treino: {loss_train.item():.4f}, Erro Validação:
num_epochs = 5000
train_losses = np.zeros(num_epochs)
val_losses = np.zeros(num_epochs)
train_network(modelo,optimizer,loss_function,x_train,y_train,x_val,y_val,num_epochs,train_losses,val_los

### **Resultados**

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(train_losses, label='Erro de Treino')
plt.plot(val_losses, label='Erro de Validação')
plt.legend()
plt.show()

In [ ]:
predictions_train = []
predictions_val = []
predictions_test = []
with torch.no_grad():
predictions_train = modelo(x_train)
predictions_val = modelo(x_val)
predictions_test = modelo(x_test)

In [ ]:
# Cálculo do erro (Função de Custo)
erro_train = loss_function(predictions_train,y_train)
erro_val = loss_function(predictions_val,y_val)
erro_test = loss_function(predictions_test,y_test)
print(f"Erro de Treino: {erro_train}")
print(f"Erro de Validação: {erro_val}")
print(f"Erro de Teste: {erro_test}")
Erro de Treino: 0.02001219242811203
Erro de Validação: 0.008439882658421993
Erro de Teste: 0.007400526665151119
# Cálculo da Acurácia de Classificação:
pred_train = torch.argmax(predictions_train, dim=1)
label_train = torch.argmax(y_train, dim=1)
pred_val = torch.argmax(predictions_val, dim=1)
label_val = torch.argmax(y_val, dim=1)
pred_test = torch.argmax(predictions_test, dim=1)
label_test = torch.argmax(y_test, dim=1)
from sklearn.metrics import accuracy_score
acc_train = accuracy_score(label_train, pred_train)
acc_val = accuracy_score(label_val, pred_val)
acc_test = accuracy_score(label_test, pred_test)
print(f"Acurácia de Treino: {acc_train*100:.2f}%")
print(f"Acurácia de Validação: {acc_val*100:.2f}%")
print(f"Acurácia de Teste: {acc_test*100:.2f}%")

### **Problema XOR**
### **Modelo**


In [ ]:
# torch.manual_seed(7) # aprende as portas Not-AND e Not-OR
torch.manual_seed(9) # aprende as portas AND e OR
class MLPXor(nn.Module):
def init (self):
super(MLPXor,self). init ()
self.hidden = nn.Linear(2, 2)
self.output = nn.Linear(2, 1)
def forward(self,x):
hidden = self.hidden(x)
x1 = torch.sigmoid(hidden)
output = self.output(x1)
x2 = torch.sigmoid(output)
return x2, x1
modeloXOR = MLPXor() # Criação do modelo (rede)
MLPXor(
(hidden): Linear(in_features=2, out_features=2, bias=True)
(output): Linear(in_features=2, out_features=1, bias=True)
)

### **Otimizador e Função de Custo**

In [ ]:
loss_function = nn.MSELoss()
# optimizer = torch.optim.Adam(modeloXOR.parameters(),lr=eta)
optimizer = torch.optim.SGD(modeloXOR.parameters(),lr=0.02, momentum=0.9)

### **Dados (conjunto XOR)**

In [ ]:
x_xor = torch.Tensor([[0,0],[0,1], [1,0], [1,1]])
y_xor = torch.Tensor([0,1,1,0]).view(-1,1)
print(x_xor)
print(y_xor)


### **Treinamento**

In [ ]:
from torch.autograd import Variable
def train_xor(model,optimizer,loss_function,x_train,y_train,num_epochs,train_losses):
for epoch in range(num_epochs):
for j in range(4):
exemplo = np.random.randint(4)
x = Variable(x_xor[exemplo], requires_grad=False)
y = Variable(y_xor[exemplo], requires_grad=False)
optimizer.zero_grad()
y_hat, _ = modeloXOR(x)
loss_train = loss_function.forward(y_hat, y)
loss_train.backward()
optimizer.step()
train_losses[epoch] = loss_train.item()
if (epoch + 1) % 1000 == 0:
print(f"Epoch {epoch+1}/{num_epochs}, Erro Treino: {loss_train.item():.4f}")
num_epochs = 5000
train_losses = np.zeros(num_epochs)
train_xor(modeloXOR,optimizer,loss_function,x_xor,y_xor,num_epochs,train_losses)

### **Resultados**

In [ ]:
predictions, hidden = modeloXOR(x_xor)
pred = predictions > 0.5
hid = hidden > 0.5
print(x_xor)
print(predictions)
print(hidden)


In [ ]:
print("Saídas do modelo: ")
print(pred)
print("Saídas das Camadas Ocultas: ")
print(hid)
Saídas do modelo:
tensor([[False],
[ True],
[ True],
[False]])
Saídas das Camadas Ocultas:
tensor([[False, False],
[False, True],
[False, True],
[ True, True]])

Analisando os resultados acima, podemos ver, pela variável hid, que ilustra os valores dos dois neurônios ocultos, que o neurônio H1 representa uma porta AND (E) e o neurônio H2 representa uma porta OR (OU)

É importante destacar que outras con gurações de portas podem ser obtidas, por exemplo, a porta Not-AND e Not-OR. A condição inicial do modelo de ne as portas que o modelo irá aprender.

**Exercício de Apoio**

Realizar as seguintes atividades:
1. Treinar o modelo com outras con guração (variar o número de camadas e neurônios
2. Avaliar o processo de treinamento considerando outros valores para a taxa de aprendizagem

# **Exercício de Apoio - Semana 03 (Parte 02)**

### **Rede Neural Multilayer Perceptron MLP (Regressão)**

* Carregamento dos pacotes
* Necessários Geração dos dados
* Definição do modelo
* Treinamento de modelo
* Gráficos e Avaliação
* Exercício de Apoio

### **Pacotes**

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import sklearn.datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

### **Dados**
### **Geração dos Dados**

In [ ]:
X = np.arange(0,7,0.05)
Y = np.sin(X) + 0.3*np.random.randn(X.shape[0]) + 0.5*X
plt.plot(X,Y, '.')
plt.show()
print(X.shape)

### **Separação Treino/Validação/Teste**

In [ ]:
X = X.reshape((-1, 1))
Y = Y.reshape((-1, 1))
# Separação desenvolvimento (30) e teste (70)
X_dev, X_test, Y_dev, Y_test = train_test_split(X, Y, test_size=0.8, random_state=42)
# Separação treino (80) e validação (20)
X_train, X_val, Y_train, Y_val = train_test_split(X_dev, Y_dev, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
plt.plot(X_train, Y_train, '*', label='Treino')
plt.plot(X_val, Y_val, '.', label='Validação')
plt.plot(X_test, Y_test, '.', label='Teste')
plt.legend()
plt.show()


**Transformação dos dados em tensores Pytorch**

In [ ]:
x_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(Y_train)
x_val = torch.FloatTensor(X_val)
y_val = torch.FloatTensor(Y_val)
x_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(Y_test)
# verificando disponibilidade da gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
device(type='cpu')


### **Modelo com uma única camada (linear)**

In [ ]:
class MLPLin(nn.Module):
def init (self,input_dim,output_dim):
super(MLPLin,self). init ()
self.rede = nn.Sequential(
nn.Linear(input_dim,output_dim),
)
def forward(self,x):
out = self.rede(x)
return out

### **Instanciação do Modelo**

In [ ]:
input_dim = 1 # número de atributos do Iris
output_dim = 1 # número de classes
modeloLin = MLPLin(input_dim,output_dim)
from torchsummary import summary
print(modeloLin)
summary(modeloLin, (100,1))
MLPLin(
(rede): Sequential(
(0): Linear(in_features=1, out_features=1, bias=True)
)
)


### **Otimizador e Função de Custo**

In [ ]:
eta = 0.02
loss_function = nn.MSELoss()
optimizer = torch.optim.SGD(modeloLin.parameters(),lr=eta)

### **Treinamento**
### **Laço de treinamento da rede**

In [ ]:
def train_network(model,optimizer,loss_function,x_train,y_train,x_val,y_val,num_epochs,train_losses,val_
for epoch in range(num_epochs):
# zerando os gradientes da época anterior
optimizer.zero_grad()
# fase de propagação
output_train = model(x_train)
# cálculo do erro (função de custo - loss function)
loss_train = loss_function(output_train, y_train)
# fase de retroprogação
loss_train.backward()
# atualização dos pesos da rede
optimizer.step()
# avaliando o modelo com o conjunto de validação
output_val = model(x_val)
loss_val = loss_function(output_val,y_val)
train_losses[epoch] = loss_train.item()
val_losses[epoch] = loss_val.item()
if (epoch + 1) % 1000 == 0:
print(f"Epoch {epoch+1}/{num_epochs}, Erro Treino: {loss_train.item():.4f}, Erro Validação:
num_epochs = 5000
train_losses = np.zeros(num_epochs)
val_losses = np.zeros(num_epochs)
train_network(modeloLin,optimizer,loss_function,x_train,y_train,x_val,y_val,num_epochs,train_losses,val_

### **Resultados**

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(train_losses, label='Erro de Treino')
plt.plot(val_losses, label='Erro de Validação')
plt.legend()
plt.show()

In [ ]:
predictions_train = []
predictions_val = []
predictions_test = []
with torch.no_grad():
predictions_train = modeloLin(x_train)
predictions_val = modeloLin(x_val)
predictions_test = modeloLin(x_test)
erro_train = loss_function(predictions_train,y_train)
erro_val = loss_function(predictions_val,y_val)
erro_test = loss_function(predictions_test,y_test)
# from sklearn.metrics import accuracy_score
# acc_train = accuracy_score(x_train, predictions_train)
print(f"Erro de Treino: {erro_train}")
print(f"Erro de Validação: {erro_val}")
print(f"Erro de Teste: {erro_test}")
Erro de Treino: 0.4135316014289856
Erro de Validação: 0.5751857757568359
Erro de Teste: 0.3441910445690155
plt.plot(x_train, y_train, 'x', label='Dados de Treino')
plt.plot(x_train, predictions_train, '.', label='Valores Preditos')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

In [ ]:
plt.plot(x_test, y_test, '.', label='Dados de Teste')
plt.plot(x_test, predictions_test, '.', label='Valores Preditos')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

In [ ]:
plt.plot(y_test, predictions_test, '.')
plt.xlabel('y (real)')
plt.ylabel('y (predito)')
# plt.legend()
plt.show()

### **Modelo com várias camadas**


In [ ]:
class MLP(nn.Module):
def init (self,input_dim,output_dim):
super(MLP,self). init ()
self.rede = nn.Sequential(
nn.Linear(input_dim,512),
nn.ReLU(),
nn.Linear(512,256),
nn.ReLU(),
nn.Linear(256,128),
nn.ReLU(),
nn.Linear(128,64),
nn.ReLU(),
nn.Linear(64,output_dim),
)
def forward(self,x):
out = self.rede(x)
return out

### **Instanciação do Modelo**

In [ ]:
input_dim = 1 # número de atributos do Iris
output_dim = 1 # número de classes
modelo = MLP(input_dim,output_dim)
from torchsummary import summary
print(modelo)
summary(modelo, (100,1))
MLP(
(rede): Sequential(
(0) : Linear(in_features=1, out_features=512, bias=True)
(1) : ReLU()
(2) : Linear(in_features=512, out_features=256, bias=True)
(3) : ReLU()
(4) : Linear(in_features=256, out_features=128, bias=True)
(5) : ReLU()
(6) : Linear(in_features=128, out_features=64, bias=True)
(7) : ReLU()(8) : Linear(in_features=64, out_features=1, bias=True)
)
)


### **Otimizador e Função de Custo**

In [ ]:
eta = 0.02
loss_function = nn.MSELoss()
# loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(modelo.parameters(),lr=eta)

### **Treinamento**
### **Transformação dos dados em tensores Pytorch**

In [ ]:
x_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(Y_train)
x_val = torch.FloatTensor(X_val)
y_val = torch.FloatTensor(Y_val)
x_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(Y_test)
# verificando disponibilidade da gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
device(type='cpu')

### **Laço de treinamento da rede**


In [ ]:
def train_network(model,optimizer,loss_function,x_train,y_train,x_val,y_val,num_epochs,train_losses,val_
for epoch in range(num_epochs):
# zerando os gradientes da época anterior
optimizer.zero_grad()
# fase de propagação

In [ ]:
output_train = model(x_train)
# cálculo do erro (função de custo - loss function)
loss_train = loss_function(output_train, y_train)
# fase de retroprogação
loss_train.backward()
# atualização dos pesos da rede
optimizer.step()
# avaliando o modelo com o conjunto de validação
output_val = model(x_val)
loss_val = loss_function(output_val,y_val)
train_losses[epoch] = loss_train.item()
val_losses[epoch] = loss_val.item()
if (epoch + 1) % 1000 == 0:
print(f"Epoch {epoch+1}/{num_epochs}, Erro Treino: {loss_train.item():.4f}, Erro Validação:
num_epochs = 20000
train_losses = np.zeros(num_epochs)
val_losses = np.zeros(num_epochs)
train_network(modelo,optimizer,loss_function,x_train,y_train,x_val,y_val,num_epochs,train_losses,val_los

### **Resultados**

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(train_losses, label='Erro de Treino')
plt.plot(val_losses, label='Erro de Validação')
plt.legend()
plt.show()

In [ ]:
predictions_train = []
predictions_val = []
predictions_test = []
with torch.no_grad():
predictions_train = modelo(x_train)
predictions_val = modelo(x_val)
predictions_test = modelo(x_test)
erro_train = loss_function(predictions_train,y_train)
erro_val = loss_function(predictions_val,y_val)
erro_test = loss_function(predictions_test,y_test)
# from sklearn.metrics import accuracy_score
# acc_train = accuracy_score(x_train, predictions_train)
print(f"Erro de Treino: {erro_train}")
print(f"Erro de Validação: {erro_val}")
print(f"Erro de Teste: {erro_test}")
Erro de Treino: 0.011329781264066696
Erro de Validação: 0.0855075791478157
Erro de Teste: 0.13274773955345154
plt.plot(x_train, y_train, 'x', label='Dados de Treino')
plt.plot(x_train, predictions_train, '.', label='Valores Preditos')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

In [ ]:
plt.plot(x_test, y_test, '.', label='Dados de Teste')
plt.plot(x_test, predictions_test, '.', label='Valores Preditos')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

In [ ]:
plt.plot([0,torch.max(y_test)],[0,torch.max(y_test)], 'black')
plt.plot(y_test, predictions_test, '.')
plt.xlabel('y (real)')
plt.ylabel('y (predito)')
# plt.legend()
plt.show()

### **Exercício de Apoio (Regressão)**
### **Realizar as seguintes atividades:**
1. Treinar o modelo com outras con guração (variar o número de camadas e neurônios
2. Avaliar o processo de treinamento considerando outros valores para a taxa de aprendizagem
3. Avaliar outras funções de ativação:
* Sigmoid Logística
* (nn.Sigmoid) Relu (nn.ReLU)